In [ ]:
import cv2
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
import numpy as np 

In [ ]:
import os
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import Accuracy, MeanIoU
# from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

In [ ]:
def ImageSegmentationModel(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Middle
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    up1 = UpSampling2D(size=(2, 2))(conv2)

    # Decoder
    concat1 = Concatenate()([conv1, up1])
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(concat1)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(conv3)

    # Output Layer (Binary Mask)
    output = Conv2D(1, 1, activation='sigmoid')(conv3)

    model = Model(inputs=inputs, outputs=output)
    return model


In [ ]:
input_shape = (256, 256, 3)
model = ImageSegmentationModel(input_shape)
model.compile(optimizer=Adam(lr=1e-4), loss=binary_crossentropy, metrics=[Accuracy(), MeanIoU(num_classes=2)])
model.summary()

In [ ]:
inputDir = 'InputDataset'
input_files = [f for f in os.listdir(inputDir) if os.path.isfile(os.path.join(inputDir, f))]

outputDir = 'OutputDataset'
output_files = [f for f in os.listdir(outputDir) if os.path.isfile(os.path.join(outputDir, f))]

In [ ]:
input_files = sorted(input_files)
output_files = sorted(output_files)

In [ ]:
# Creating a matrix/tensor for the image dataset
X = []
y = []

for input, output in zip(input_files, output_files):
    image = cv2.imread(os.path.join(inputDir, input))
    image = image/255
    mask = cv2.imread(os.path.join(outputDir, output))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = 1 - (mask/255)
    X.append(image)
    y.append(mask)

In [ ]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=20, callbacks=[checkpoint])